# Install dependencies

In [ ]:
! pip install pandas transformers datasets scikit-learn simpletransformers torch tqdm

# Initialize packages and data

In [1]:
import pandas as pd
import torch
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
from tqdm import tqdm

tqdm.pandas()

train_stances = pd.read_csv('train_stances.csv', index_col="Body ID")
train_bodies = pd.read_csv('train_bodies.csv', index_col="Body ID")
test_stances = pd.read_csv('competition_test_stances.csv', index_col="Body ID")
test_bodies = pd.read_csv('competition_test_bodies.csv', index_col="Body ID")

# Choose model

In [10]:
# Decide on a model from this list
# https://huggingface.co/models?search=pegasus
model_name = "google/pegasus-multi_news"

# Initialize model

In [12]:
torch.set_num_threads(8)
device = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name)

def summarize(src_text):
    batch = tokenizer(src_text, truncation=True, padding="longest", return_tensors="pt").to(device)
    translated = model.generate(**batch)
    tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
    return tgt_text[0]

[W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)


# Run model

In [ ]:
train_bodies["articleBody"] = train_bodies["articleBody"].progress_apply(summarize)
train_bodies.to_csv(f'train_bodies_{model_name.replace("/","_")}.csv')

 26%|█████████▎                          | 437/1683 [2:20:29<7:42:14, 22.26s/it]

In [ ]:
test_bodies["articleBody"] = test_bodies["articleBody"].progress_apply(summarize)
test_bodies.to_csv(f'test_bodies_{model_name.replace("/","_")}.csv')